# Distance Computation

A script to compute the distance from each hand-joint to the object, based on the pose data.


#### Notes

Input:

- Hand Pose: 1 (annotate?) + $21*3$ (x,y,z in order) (left hand) + 1 + $21*3$ (right hand)
- Object Pose: 1 (object class) + $21*3$ (x,y,z in order) bounding box (1 center, 8 corners, 12 mid-edge)

Output:

- Distance map: 21 (left hand) + 21 (right hand) distances from joints

----

Imports

In [55]:
import os
import numpy as np
from numpy.typing import NDArray
import tqdm
from typing import List

Parameters

In [2]:
n_frames_per_seq : int = 8
mode = 'train' # 'train' , 'val' , 'test'
h2o_root = '../data/h2o/'
sample_root = h2o_root + f'seq_{n_frames_per_seq}_{mode}/'

### Data Format Helpers

In [3]:
def format_hand_poses(hand_poses: NDArray) -> NDArray:
    """
    Format the raw hand_poses vector into a 21x3 matrix for each hand

    Args:
    -----
    hand_poses: NDArray (128,)
        Raw hand poses vector

    Returns:
    --------
    hand_poses_left: NDArray (21, 3)
        Hand poses matrix for left hand
    hand_poses_right: NDArray (21, 3)
        Hand poses matrix for right hand
    """
    assert hand_poses.shape == (128,), f"hand_poses should be of shape (128,) but got {hand_poses.shape}"

    hand_poses_left = hand_poses[1:64].reshape((21, 3))
    hand_poses_right = hand_poses[65:].reshape((21, 3))

    return hand_poses_left, hand_poses_right

In [4]:
def format_object_poses(obj_poses: NDArray) -> List[NDArray]:
    """
    Format and group the raw obj_poses vector into a list of the poses.

    The list has 3 elements:
    1. (3,)   NDArray of the object's center pose
    2. (8,3)  NDArray of the object's corner vertices
    3. (12,3) NDArray of the object's mid-edge vertices

    Args:
    -----
    obj_poses: NDArray (64,)
        Raw object poses vector

    Returns:
    --------
    obj_poses_list: List[NDArray]
        List of grouped object poses
    """
    assert obj_poses.shape == (64,), f"obj_poses should be of shape (64,) but got {obj_poses.shape}"

    obj_center = obj_poses[1:4]
    obj_corners = obj_poses[4:28].reshape((8, 3))
    obj_edges = obj_poses[28:].reshape((12, 3))

    return [obj_center, obj_corners, obj_edges]

### Transformation Helpers

In [5]:
def invert_transformation_matrix(T: NDArray) -> NDArray:
    """
    Invert the transformation matrix T

    Args:
    -----
    T: NDArray (4, 4)
        Transformation matrix

    Returns:
    --------
    T_inv: NDArray (4, 4)
        Inverted transformation matrix
    """
    assert T.shape == (4, 4), f"T should be of shape (4, 4) but got {T.shape}"

    R = T[:3, :3]
    t = T[:3, 3]
    T_inv = np.eye(4)
    T_inv[:3, :3] = R.T
    T_inv[:3, 3] = -R.T @ t

    return T_inv

In [16]:
def to_hom_3D(points: NDArray) -> NDArray:
    """
    Convert the input points to homogeneous coordinates.

    Args:
    -----
    points: NDArray (N, 3)
        Input points
    
    Returns:
    --------
    points_hom: NDArray (N, 4)
        Homogeneous coordinates of the input points
    """
    assert points.shape[1] == 3, f"points should be of shape (N, 3) but got {points.shape}"
    return np.hstack((points, np.ones((points.shape[0], 1))))


In [17]:
def to_cart_3D(points_hom: NDArray) -> NDArray:
    """
    Convert the input points in homogeneous coordinates to cartesian coordinates.

    Args:
    -----
    points_hom: NDArray (N, 4)
        Input points in homogeneous coordinates
    
    Returns:
    --------
    points: NDArray (N, 3)
        Cartesian coordinates of the input points
    """
    assert points_hom.shape[1] == 4, f"points_hom should be of shape (N, 4) but got {points_hom.shape}"
    return points_hom[:, :3] / points_hom[:, 3, None]

In [7]:
def transfrom_pts(pts_hom: NDArray, T: NDArray) -> NDArray:
    """
    Transform the input points using the input transformation matrix.

    Args:
    -----
    pts_hom: NDArray (N, 4)
        Input points in homogeneous coordinates
    T: NDArray (4, 4)
        Transformation matrix
    
    Returns:
    --------
    pts_transformed_hom: NDArray (N, 4)
        Transformed points in homogeneous coordinates
    """
    assert pts_hom.shape[1] == 4, f"pts should be of shape (N, 4) but got {pts_hom.shape}"
    assert T.shape == (4, 4), f"T should be of shape (4, 4) but got {T.shape}"

    return np.dot(T, pts_hom.T).T

In [23]:
def assert_box_transform(T_OC: NDArray, C_obj_center: NDArray):
    """
    Asserts the transformation from the camera to the object frame is correct.

    Args:
    -----
    T_OC: NDArray (4, 4)
        Transformation matrix from the camera to the object frame
    C_obj_center: NDArray (3,)
        Object's center in the camera frame
    """
    assert T_OC.shape == (4, 4), f"T_OC should be of shape (4, 4) but got {T_OC.shape}"
    assert C_obj_center.shape == (3,), f"C_obj_center should be of shape (3,) but got {C_obj_center.shape}"

    O_obj_center = transfrom_pts(to_hom_3D(C_obj_center[np.newaxis, :]), T_OC)
    # assert this is close to [0, 0, 0, 1]
    assert np.allclose(O_obj_center, np.array([[0, 0, 0, 1]]), atol=1e-6), \
        f"Object center transformation failed. Got {O_obj_center}, expected [0, 0, 0, 1]."

### BBox Helpers

In [57]:
def get_bbox_dim_from_object(O_obj_corners: NDArray) -> NDArray:
    """
    Get the dimensions of the object's bounding box in the object's local coordinate frame.

    Computes the dimensions through the median of the object's corner vertices
    in the local coordinate frame.

    Args:
    -----
    O_obj_corners: NDArray (8, 3)
        Object's corner vertices
    
    Returns:
    --------
    box_dim: NDArray (3,)
        Dimensions of the object's bounding box in the local coordinate frame.
    """
    assert O_obj_corners.shape == (8, 3), f"O_obj_corners should be of shape (8, 3) but got {O_obj_corners.shape}"

    O_obj_corners = np.abs(O_obj_corners) # ignore direction: take the absolute value of all elements
    box_dim = np.median(O_obj_corners, axis=0) # take the median along all axes
    return 2*box_dim

In [58]:
def get_bbox_dim_from_camera(T_OC: NDArray, C_obj_corners):
    """
    Compute the bounding box dimensions of the object in the camera coordinate frame.

    Args:
    -----
    T_OC: NDArray (4, 4)
        Transformation matrix from camera to object coordinate frame
    C_obj_corners: NDArray (8, 3)
        Object's corner vertices in the camera coordinate frame
    
    Returns:
    --------
    box_dim: NDArray (3,)
        Dimensions of the object's bounding box in the camera coordinate frame.
    """
    assert T_OC.shape == (4, 4), f"T_OC should be of shape (4, 4) but got {T_OC.shape}"
    assert C_obj_corners.shape == (8, 3), f"C_obj_corners should be of shape (8, 3) but got {C_obj_corners.shape}"

    # transform object corners to object coordinate frame
    O_obj_corners = transfrom_pts(to_hom_3D(C_obj_corners), T_OC)
    O_obj_corners = to_cart_3D(O_obj_corners)
    # get object dimensions
    return get_bbox_dim_from_object(O_obj_corners)

### Geometry Helpers

In [81]:
def line_plane_intersection(line_pt: NDArray, line_dir: NDArray, plane_vertices: NDArray):
    """
    Find the intersection, if any, of a line and a plane defined by 4 vertices.

    Parameters
    ----------
    line_pt : NDArray (3,)
        A point on the line.
    line_dir : NDArray (3,)
        The direction of the line.
    plane_vertices : NDArray (4,3)
        4 vertices defining the plane.
    
    Returns
    -------
    NDArray (3,) or None
        The intersection point or None if the line is parallel to the plane.
    """
    assert line_pt.shape == (3,)
    assert line_dir.shape == (3,)
    assert plane_vertices.shape == (4,3)

    plane_normal = np.cross(plane_vertices[1] - plane_vertices[0], plane_vertices[3] - plane_vertices[0])
    plane_norm = np.linalg.norm(plane_normal)
    assert plane_norm > 1e-6
    plane_normal /= plane_norm
    
    denom = np.dot(line_dir, plane_normal)
    if denom < 1e-15:
        # debug print TODO
        # print("| No I |", end="")
        return None # line is parallel to plane
    
    t = np.dot(plane_vertices[0] - line_pt, plane_normal) / denom
    intersection = line_pt + t * line_dir
    assert intersection.shape == (3,)
    return intersection


In [69]:
def is_in_face_object(bbox_dim: NDArray, O_pt: NDArray) -> bool:
    assert bbox_dim.shape == (3,), f"bbox_dim should be of shape (3,) but got {bbox_dim.shape}"
    assert O_pt.shape == (3,), f"O_pt should be of shape (3,) but got {O_pt.shape}"
    
    tol = np.double(0.1)
    bbox_r = bbox_dim/2

    # get absolute values
    O_pt = np.abs(O_pt)
    # check that each coordinate is within the box radii
    return np.all(O_pt <= bbox_r + tol)

## New Implementation

Preparations

In [53]:
# Load number of sequences for selected mode
sample_ids = np.load(sample_root + 'sample_ids.npy')
n_sequences = sample_ids.shape[0]
print(f"Loaded {n_sequences} action sequences for {mode}.")

# path handling
hand_dir = sample_root + f'poses_hand_{mode}/'
obj_poses_dir = sample_root + f'poses_obj_{mode}/'
obj_rt_dir = sample_root + f'obj_rt_{mode}/'
dist_dir = sample_root + f'distances_{mode}/'
os.makedirs(dist_dir, exist_ok=True) # create the destination directory

# face indices according to h2o convention, but 0-indexed
faces_indices = [
        [0, 1, 2, 3], [4, 5, 6, 7], [0, 1, 5, 4], 
        [2, 3, 7, 6], [0, 3, 7, 4], [1, 2, 6, 5]
    ]

Loaded 569 action sequences for train.


In [56]:
def get_faces(obj_corners: NDArray) -> List[NDArray]:
    faces = []
    for face_indices in faces_indices:
        face = obj_corners[face_indices]
        faces.append(face)
    return faces

In [92]:
# for seq_id in range(1, n_sequences+1):
for seq_id in range(1, 14): # TODO
    print("\n\nSequence:", seq_id, '\n') # TODO
    n_dist_not_found = 0 # TODO

    # get poses of current sequence
    hand_poses = np.load(hand_dir + f'{seq_id:03d}.npy')
    obj_poses = np.load(obj_poses_dir + f'{seq_id:03d}.npy')
    obj_rts = np.load(obj_rt_dir + f'{seq_id:03d}.npy')
    assert hand_poses.shape == (n_frames_per_seq, 128)
    assert obj_poses.shape == (n_frames_per_seq, 64)
    assert obj_rts.shape == (n_frames_per_seq, 4, 4)

    for frame_id in range(n_frames_per_seq):
        hand_poses_left, hand_poses_right = format_hand_poses(hand_poses[frame_id])
        obj_center, obj_corners, obj_mid_edge = format_object_poses(obj_poses[frame_id])
        T_CO = obj_rts[frame_id]
        T_OC = invert_transformation_matrix(T_CO)

        assert_box_transform(T_OC, obj_center)

        # get object dimensions
        box_dim = (get_bbox_dim_from_camera(T_OC, obj_corners))
        # print("Object dimensions:", box_dim)

        # get faces
        faces = get_faces(obj_corners)
        # transfrom faces to object frame
        for face in faces:
            face = transfrom_pts(to_hom_3D(face), T_OC)
            face = to_cart_3D(face)

        # compute intersections
        for j in range(21):
            # TODO
            inters = []


            min_dist = 10.0
            found_dist = False

            joint = hand_poses_left[j]

            # transform joint to object frame
            joint = transfrom_pts(to_hom_3D(joint[np.newaxis, :]), T_OC)
            joint = to_cart_3D(joint)[0]

            # compute the line to the object's center
            center_vec = joint
            center_vec_norm = np.linalg.norm(center_vec)
            assert center_vec_norm > 1e-6
            center_vec /= center_vec_norm

            # debug print TODO
            # print("Computing distance for a joint...")
            # in_square_count = 0

            for face in faces:
                # compute the intersection point
                inter = line_plane_intersection(np.array([0.0, 0.0, 0.0], dtype=np.float64), center_vec, face)
                inters.append(inter)

                if inter is None:
                    continue

                # transform intersection point to object frame
                # inter = transfrom_pts(to_hom_3D(inter[np.newaxis, :]), T_OC)
                # inter = to_cart_3D(inter)[0]
                # print("Inter:", inter)

                # check if the intersection point is within the face
                if is_in_face_object(box_dim, inter):
                    # debug counter TODO
                    # in_square_count += 1

                    # compute the distance
                    dist = np.linalg.norm(inter - joint)
                    if dist < min_dist:
                        min_dist = dist
                    found_dist = True

            if not found_dist:
                print("No distance found:")
                print("Distance to center:", center_vec_norm)
                # print(inters)
                # print("Object center:", obj_center)
                # print("Joint:", joint)
                n_dist_not_found += 1
                # TODO
                # print("\n")
            
            
            # debug print TODO
            # print(f"Found {in_square_count} intersections in square.")
            
            # assert found_dist, "No distance found."
            # if found_dist: 
            #     return min_dist
            # else:
            #     return None

    print("Number of distances not found:", n_dist_not_found) # TODO



Sequence: 1 

No distance found:
Distance to center: 0.2962043492977741
No distance found:
Distance to center: 0.2605344505317075
No distance found:
Distance to center: 0.23022317987624755
No distance found:
Distance to center: 0.20499729976837214
No distance found:
Distance to center: 0.18239906002769662
No distance found:
Distance to center: 0.21274075133876746
No distance found:
Distance to center: 0.1855195039892929
No distance found:
Distance to center: 0.1794090081368972
No distance found:
Distance to center: 0.1747185542650982
No distance found:
Distance to center: 0.22102921631727165
No distance found:
Distance to center: 0.19702300862710057
No distance found:
Distance to center: 0.18712956843997408
No distance found:
Distance to center: 0.18106206022572877
No distance found:
Distance to center: 0.2439884581028264
No distance found:
Distance to center: 0.2196249840819052
No distance found:
Distance to center: 0.20526947625754347
No distance found:
Distance to center: 0.195128

## Old Implementation

Load Pose Sequences

In [60]:
action_labels = np.load(h2o_root + f'action_labels_{mode}.npy')
n_sequences = action_labels.shape[0]
print(f"Loaded {n_sequences} action sequences for {mode}.")

Loaded 569 action sequences for train.


In [65]:
def line_plane_intersection(line_pt: NDArray, line_dir: NDArray, plane_vertices: NDArray):
    """
    Find the intersection, if any, of a line and a plane defined by 4 vertices.

    Parameters
    ----------
    line_pt : NDArray (3,)
        A point on the line.
    line_dir : NDArray (3,)
        The direction of the line.
    plane_vertices : NDArray (4,3)
        4 vertices defining the plane.
    
    Returns
    -------
    NDArray (3,) or None
        The intersection point or None if the line is parallel to the plane.
    """
    assert line_pt.shape == (3,)
    assert line_dir.shape == (3,)
    assert plane_vertices.shape == (4,3)

    plane_normal = np.cross(plane_vertices[1] - plane_vertices[0], plane_vertices[3] - plane_vertices[0])
    plane_norm = np.linalg.norm(plane_normal)
    assert plane_norm > 1e-6
    plane_normal /= plane_norm
    
    denom = np.dot(line_dir, plane_normal)
    if denom < 1e-12:
        # debug print TODO
        # print("| No I |", end="")
        return None # line is parallel to plane
    
    t = np.dot(plane_vertices[0] - line_pt, plane_normal) / denom
    intersection = line_pt + t * line_dir
    assert intersection.shape == (3,)
    return intersection


In [95]:
def is_pt_in_rect(pt: NDArray, rect_vertices: NDArray) -> bool:
    """
    Check whether a point is within a rectangle defined by the 4 vertices.

    Parameters
    ----------
    pt : NDArray (3,)
        The point to check.
    rect_vertices : NDArray (4, 3)
        The vertices of the rectangle, such that succeeding vertices are connected.

    Returns
    -------
    bool
        Whether the point is within the rectangle.


    Algorithm:
    Take 3 consecutive vertices ABC of the rectangle and the sample point P.
    P is inside the rectangle, iff 
    0 <= dot(AB, AP) <= dot(AB, AB) and 
    0 <= dot(BC, BP) <= dot(BC, BC)
    """
    assert pt.shape == (3,)
    assert rect_vertices.shape == (4, 3)
    tol = np.double(1.0)

    # compute AB, BC
    edge1 = rect_vertices[1] - rect_vertices[0]
    edge2 = rect_vertices[2] - rect_vertices[1]
    # compute AP, BP
    vec1 = pt - rect_vertices[0]
    vec2 = pt - rect_vertices[1]

    return -tol <= np.dot(edge1, vec1) <= np.dot(edge1, edge1) + tol and -tol <= np.dot(edge2, vec2) <= np.dot(edge2, edge2) + tol


    # dennis
    # total_angle = 0
    # for i in range(rect_vertices.shape[0]):
    #     a, b = rect_vertices[i], rect_vertices[(i + 1) % len(rect_vertices)]
    #     da, db = a - pt, b - pt
    #     angle = np.arctan2(np.linalg.norm(np.cross(da, db)), np.dot(da, db))
    #     total_angle += angle
    # return np.isclose(total_angle, 2 * np.pi, atol=1e-5)

    

In [63]:
def dist_joint_bbox(joint: NDArray, center: NDArray, faces: NDArray):
    """
    Compute the distance from a joint to the bounding box of an object.

    Parameters
    ----------
    joint : NDArray (3,)
        The joint position.
    center : NDArray (3,)
        The object's center.
    faces : List[NDArray (4,3)]
        The faces of the object's bounding box.
    
    Returns
    -------
    float
        The minimum distance from the joint to the bounding box.

    Exceptions
    ----------
    AssertionError
        If no distance is found.
    """
    assert joint.shape == (3,)
    assert center.shape == (3,)
    assert faces[0].shape == (4, 3)
    
    min_dist = 10.0
    found_dist = False

    # compute the line to the object's center
    center_vec = center - joint
    center_vec_norm = np.linalg.norm(center_vec)
    assert center_vec_norm > 1e-6
    center_vec /= center_vec_norm

    # debug print TODO
    # print("Computing distance for a joint...")
    # in_square_count = 0

    for face in faces:
        # compute the intersection point
        inter = line_plane_intersection(joint, center_vec, face)

        # check if the intersection point is within the face
        if inter is not None and is_pt_in_rect(inter, face):
            # debug counter TODO
            # in_square_count += 1

            # compute the distance
            dist = np.linalg.norm(inter - joint)
            if dist < min_dist:
                min_dist = dist
            found_dist = True
    
    # debug print TODO
    # print(f"Found {in_square_count} intersections in square.")
    
    # assert found_dist, "No distance found."
    if found_dist: 
        return min_dist
    else:
        return None

    # return min_dist

In [96]:
# path handling
pose_root = h2o_root + f'seq_{n_frames_per_seq}_{mode}/'
hand_dir = pose_root + f'poses_hand_{mode}/'
obj_dir = pose_root + f'poses_obj_{mode}/'
dist_dir = pose_root + f'distances_{mode}/'
# os.makedirs(dist_dir, exist_ok=True) # create the destination directory

# face indices according to h2o convention, but 0-indexed
faces_indices = [
        [0, 1, 2, 3], [4, 5, 6, 7], [0, 1, 5, 4], 
        [2, 3, 7, 6], [0, 3, 7, 4], [1, 2, 6, 5]
    ]

# for each sequence TODO
for seq_id in range(1, n_sequences+1):
    print("\n\nSequence:", seq_id, '\n') # TODO
    no_dist = 0

    # get poses of current sequence
    hand_poses = np.load(hand_dir + f'{seq_id:03d}.npy')
    obj_poses = np.load(obj_dir + f'{seq_id:03d}.npy')
    assert hand_poses.shape[0] == n_frames_per_seq
    assert hand_poses.shape[1] == 128
    assert obj_poses.shape[0] == n_frames_per_seq
    assert obj_poses.shape[1] == 64

    # prepare distance array
    dist_array = np.zeros((n_frames_per_seq, 42))

    # for each frame TODO
    for frame_id in range(0, n_frames_per_seq):
        # print("\n\nFrame:", frame_id, '\n') # TODO
        
        # get and format hand poses of current frame
        hand_poses_left = hand_poses[frame_id, 1:64].reshape((21, 3))
        hand_poses_right = hand_poses[frame_id, 65:].reshape((21, 3))

        # extract object center point
        obj_center = obj_poses[frame_id, 1:4]
        # extract object corner vertices
        obj_corner_vert = obj_poses[frame_id, 4:]
        obj_corner_vert = obj_corner_vert[:8*3]
        obj_corner_vert = obj_corner_vert.reshape((8, 3))
        # extract faces as (4,3) arrays from the corner vertices according to the face indices
        faces = []
        for face_indices in faces_indices:
            face = obj_corner_vert[face_indices]
            faces.append(face)

        # for each joint
        for j in range(21):
            # print("\nJoint:", j) # TODO
            # compute the line to the object's center
            # print("\nLeft") # TODO
            distance_left = dist_joint_bbox(hand_poses_left[j], obj_center, faces)
            # print("\nRight") # TODO
            # print("Point:", hand_poses_right[j]) # TODO
            # print("BBox", faces) # TODO
            distance_right = dist_joint_bbox(hand_poses_right[j], obj_center, faces)
            # print("Distance:", distance_right) # TODO

            if distance_left is None:
                no_dist += 1
                print("No dist in frame", frame_id, "for left join", j)
            if distance_right is None:
                no_dist += 1
                print("No dist in frame", frame_id, "for right join", j)

            # write distance to array TODO
            # dist_array[frame_id, j] = distance_left
            # dist_array[frame_id, j+21] = distance_right

    # TODO 
    print("No distances found:", no_dist)
    # save the distance array

# output array
# for each sequence a file
# array (n_sequences, 42)




Sequence: 1 

No distances found: 0


Sequence: 2 

No distances found: 0


Sequence: 3 

No distances found: 0


Sequence: 4 

No distances found: 0


Sequence: 5 

No distances found: 0


Sequence: 6 

No dist in frame 2 for right join 13
No distances found: 1


Sequence: 7 

No distances found: 0


Sequence: 8 

No distances found: 0


Sequence: 9 

No distances found: 0


Sequence: 10 

No dist in frame 0 for left join 10
No dist in frame 0 for left join 11
No dist in frame 1 for left join 14
No dist in frame 2 for left join 14
No dist in frame 3 for left join 12
No dist in frame 3 for left join 15
No dist in frame 4 for left join 11
No dist in frame 4 for left join 12
No distances found: 8


Sequence: 11 

No dist in frame 2 for left join 12
No dist in frame 5 for left join 10
No dist in frame 5 for left join 16
No distances found: 3


Sequence: 12 

No distances found: 0


Sequence: 13 

No distances found: 0


Sequence: 14 

No distances found: 0


Sequence: 15 

No distances 

KeyboardInterrupt: 

In [32]:
arr = np.array([1, 2, 3], dtype=np.double)
r = np.linalg.norm(arr)
print(type(r))

<class 'numpy.float64'>
